In [ ]:
import csv
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from climatenet.utils.data import ClimateDatasetLabeled, ClimateDataset
from climatenet.models import CGNet
from climatenet.utils.utils import Config
from climatenet.track_events import track_events
from climatenet.analyze_events import analyze_events
from climatenet.visualize_events import visualize_events
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report


def save_submission(predictions):
    with open('submission.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['SNo', 'Label'])
        for idx, label in enumerate(predictions, 1):
            writer.writerow([idx, label])


def initialize_config(config_path):
    try:
        config = Config(config_path)
        if 'architecture' in config and 'seed' in config:
            print("Configurations initialized successfully.")
            return config
    except Exception as e:
        print("Initialization error: %s" % e)
        return None


class ClimateAnalysisPipeline:
    def __init__(self, config_path):
        self.inference_set = None
        self.training_set = None
        self.config = initialize_config(config_path)
        self.model = self.initialize_model()

    def initialize_model(self):
        if self.config:
            try:
                model = CGNet(self.config)
                print("Model initialized successfully.")
                return model
            except Exception as e:
                print("Model initialization error: %s" % e)
        return None

    def load_csv_datasets(self, training_csv_path, inference_csv_path, _default_config):
        try:
            self.training_set = ClimateDatasetLabeled(pd.read_csv(training_csv_path), _default_config)
            self.inference_set = ClimateDataset(pd.read_csv(inference_csv_path), _default_config)
            print("Datasets successfully loaded from CSV.")
        except pd.errors.EmptyDataError:
            print("CSV files are empty.")
        except pd.errors.ParserError:
            print("Error while parsing CSV files.")
        except Exception as e:
            print("Dataset loading error: %s" % e)

    def train_and_evaluate(self):
        self.train_model()
        self.evaluate_baseline_models()
        self.track_and_analyze_events()

    def train_model(self):
        self.model.train(self.training_set)
        print("Model trained.")

    def evaluate_baseline_models(self):
        baseline_models = [
            (DummyClassifier(strategy="uniform"), "Dummy Classifier"),
            (SGDClassifier(), "Stochastic Gradient Descent"),
            (SVC(), "Support Vector Machine"),
            (RandomForestClassifier(), "Random Forest"),
            (LogisticRegression(max_iter=1000), "Logistic Regression"),
        ]

        for model, name in baseline_models:
            self.evaluate_baseline(model, name)

    def evaluate_baseline(self, model, model_name):
        model.fit(self.training_set.data, self.training_set.labels)
        predictions = model.predict(self.inference_set.data)
        print("%s Performance" % model_name)
        print(classification_report(self.inference_set.labels, predictions))

    def track_and_analyze_events(self):
        if self.inference_set is not None:
            class_masks = self.model.predict(self.inference_set)
            event_masks = track_events(class_masks)
            analyze_events(event_masks, class_masks, 'results/')
            visualize_events(event_masks, self.inference_set, 'pngs/')
        else:
            print("Inference set is not available.")

    def save_model(self, save_path):
        try:
            self.model.save_model(save_path)
            print("Model saved to %s" % save_path)
        except Exception as e:
            print("Failed to save the model: %s" % e)

    def visualize_with_seaborn(self):
        sns.heatmap(self.inference_set.corr(), annot=True, fmt=".2f")
        plt.show()


parser = argparse.ArgumentParser(description='Climate Analysis Pipeline')
default_config = '/content/drive/MyDrive/climateDoc/model/config-init.json'
default_train_csv = '/content/drive/MyDrive/climateDoc/classification-of-extreme-weather-events-udem/train.csv'
default_infer_csv = '/content/drive/MyDrive/climateDoc/classification-of-extreme-weather-events-udem/test.csv'

parser.add_argument('--config', type=str, default=default_config, help='Path to the config file')
parser.add_argument('--train_csv', type=str, default=default_train_csv, help='Path to the training set CSV')
parser.add_argument('--infer_csv', type=str, default=default_infer_csv, help='Path to the inference set CSV')
parser.add_argument('--save', type=str, required=False, help='models/')

args = parser.parse_args()

pipeline = ClimateAnalysisPipeline(args.config)
pipeline.load_csv_datasets(args.train_csv, args.infer_csv, default_config)
pipeline.train_and_evaluate()

if args.save:
    pipeline.save_model(args.save)

pipeline.visualize_with_seaborn()

print("Climate Analysis Pipeline execution complete.")
